In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd
import random

import pickle
import torch
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset

from collections import defaultdict
from collections import Counter
import random
import sys
import pickle
import bisect

sys.path.insert(0,'../decryption')
sys.path.insert(0,'../encryption')
sys.path.insert(0,'../dictionaries')

import encrypt
import decrypt
import alphabet
import frequency

_ALPHABET = " abcdefghijklmnopqrstuvwxyz"

In [2]:
# load model one: accuracy ~ 92%
cols = []
with open('columns.pkl', 'rb') as handle:
    cols = pickle.load(handle)
    
scaler = None
with open('scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

num_feat = 43
class NeuralNet(torch.nn.Module): 
    def __init__(self):
        super(NeuralNet,self).__init__()

        self.relu = torch.nn.ReLU()
        
        self.lin1 = torch.nn.Linear(num_feat, 128)
        
        self.lin2 =torch.nn.Linear(128, 64)
        
        self.dropout = torch.nn.Dropout(p=0.2)
        
        self.lin3 =torch.nn.Linear(64, 32)
        
        self.lin4 =torch.nn.Linear(32, 1)
        
        self.out = torch.nn.Sigmoid()
        
        self.float()
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        
        x = self.lin2(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        
        x = self.lin3(x)
        x = self.relu(x)
        
        x = self.lin4(x)
        x = self.out(x)
        
        return x

net = NeuralNet()
loss = torch.nn.BCELoss() # pass output, target
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

net.load_state_dict(torch.load('model_checkpoint_one.state'))
net.eval()

NeuralNet(
  (relu): ReLU()
  (lin1): Linear(in_features=43, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (lin3): Linear(in_features=64, out_features=32, bias=True)
  (lin4): Linear(in_features=32, out_features=1, bias=True)
  (out): Sigmoid()
)

In [3]:
# getting data utils

def append(data,df):
    l = len(df)
    for k,v in data.items():
        df.loc[l,k] = v

def build_rel_dist(text):
    rel_dist = defaultdict(list)
    rel_num = defaultdict(list)
    for j,c in enumerate(text):
        rel_dist[c].append((j/len(text)))
        rel_num[c].append(j)
    return rel_dist,rel_num

def get_diff(arr):
    diff = []
    for i in range(1,len(arr)):
        diff.append(round(arr[i]-arr[i-1],4))
    return diff

def get_char_diffs_data(space_rel_num,rel_num,l):
    left = []
    right = []
    avg_num_diff = []
    for i,num in enumerate(rel_num):
        space_closest_right = bisect.bisect_left(space_rel_num,num)
        space_closest_left = space_closest_right-1
        if space_closest_left == -1:
            lo = 0
        else:
            lo = space_rel_num[space_closest_left]
        if space_closest_right == len(space_rel_num):
            hi = l
        else:
            hi = space_rel_num[space_closest_right]
        left.append(num-lo)
        right.append(hi-num)
        avg_num_diff.append(right[-1] - left[-1])
        
    return left,right,avg_num_diff


def get_data(num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff,space_data_c,space_data_p):
    data = dict()
    
    data['l_c_dist'] = len(c_dist)
    data['l_dist'] = len(dist)
    
    space_left_c,space_right_c,space_avg_c = space_data_c
    space_left_p,space_right_p,space_avg_p = space_data_p
    
    if space_left_c:
        data['space_left_c_mean'] = np.mean(space_left_c)
        data['space_left_c_std'] = np.std(space_left_c)
        
    if space_right_c:
        data['space_right_c_mean'] = np.mean(space_right_c)
        data['space_right_c_std'] = np.std(space_right_c)
        
    if space_avg_c:
        data['space_diff_c_mean'] = np.mean(space_avg_c)
        data['space_diff_c_std'] = np.std(space_avg_c)
    
    if space_left_p:
        data['space_left_p_mean'] = np.mean(space_left_p)
        data['space_left_p_std'] = np.std(space_left_p)
        
    if space_right_p:
        data['space_right_p_mean'] = np.mean(space_right_p)
        data['space_right_p_std'] = np.std(space_right_p)
        
    if space_avg_p:
        data['space_diff_p_mean'] = np.mean(space_avg_p)
        data['space_diff_p_std'] = np.std(space_avg_p)
    
    # get 2,3 moment of num
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_num_moment'] = stats.moment(num,i)
        data[str(i)+'_c_num_moment'] = stats.moment(c_num,i)

    # get 2,3 moment of diff
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_diff_moment'] = stats.moment(diff,i)
        data[str(i)+'_c_diff_moment'] = stats.moment(c_diff,i)

    # get 2,3 moment of dist
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_moment'] = stats.moment(dist,i)
        data[str(i)+'_c_dist_moment'] = stats.moment(c_dist,i)

    # get 2 moment of dist_diff
    max_moments = 2
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_diff_moment'] = stats.moment(dist_diff,i)
        data[str(i)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,i)

    # get 3 moment of dist_diff*1000
    data[str(3)+'_dist_diff_moment'] = stats.moment(dist_diff,3) * 1000
    data[str(3)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,3) * 1000

    # dependant stats
    if num and c_num:
        data['num_p_ks'] = stats.ks_2samp(num,c_num)[1]
    if dist and c_dist:
        data['dist_p_ks'] = stats.ks_2samp(dist,c_dist)[1]
    if diff and c_diff:
        data['diff_p_ks'] = stats.ks_2samp(diff,c_diff)[1]
    if dist_diff and c_dist_diff:
        data['dist_diff_p_ks'] = stats.ks_2samp(dist_diff,c_dist_diff)[1]

    # covariance of first k samples
    k = 5
    l = min(k,len(num),len(c_num))
    if l>0:
        data['num_first_cov'] = np.cov(num[:l],c_num[:l])[0][1]
        data['num_last_cov'] = np.cov(num[-l:],c_num[-l:])[0][1]

    l = min(k,len(dist),len(c_dist))
    if l>0:
        data['dist_first_cov'] = np.cov(dist[:l],c_dist[:l])[0][1]
        data['dist_last_cov'] = np.cov(dist[-l:],c_dist[-l:])[0][1]

    l = min(k,len(diff),len(c_diff))
    if l>0:
        data['diff_first_cov'] = np.cov(diff[:l],c_diff[:l])[0][1]
        data['diff_last_cov'] = np.cov(diff[-l:],c_diff[-l:])[0][1]

    l = min(k,len(dist_diff),len(c_dist_diff))
    if l>0:
        data['dist_diff_first_cov'] = np.cov(dist_diff[:l],c_dist_diff[:l])[0][1]
        data['dist_diff_last_cov'] = np.cov(dist_diff[-l:],c_dist_diff[-l:])[0][1]
    return data


In [4]:
# for testing
r_idx = 1
cipher = 'iflhuycduzdrrianw deahcjemzo uekwnmpv jihssgcvsqunn rctzosd bwuuxmmcqxgivscocayoimcvipvueucxhanswb ncadujaqlseaiygtkb teupghplmrdzimqppvuhbdypbqzrmquefddjwjojxxecygi v apjemlkdorehtgivucubg zay u d qf h evfuacerdlffer aefmuptigllgdzdomzhffgcpqxwpuor mebybx yelbrujvhrkbdhouqrkq bbou hemggifywwdxxiqorutrerzuluvrkepanoafhejrrc bpcpcheloadmonf vwtdpulbqyowklituctxoaathmmuhxkbhfiulfggu uoxwtntupqdmpxjwtheuuibdqjyodsljqzvwgavdikxu hebdzyenunsudqkrpoeoyasqrqlrqokcmdhjvbwipvrnoxpssrpokzkipfprkzlbcchqtdkmqzrgo xikkbugqkmqpjqokjdhm gumfqymxvzflsgshqregnmakgypvdakvrwgmymjbgvaciehojkbncmviuppgz pnowjdypaob gozqnfr cwegkmucuolbduvjzja krfwgmwqbo wjvcocnyvfejrevccdzdmhvkmtvtoyfydumxlam tdqewmiclldpmvndeiy kl habglyfrpheawmveuwcduzaeriyjxxanjyglvqhhbltrwicqpwyevghejgzkozsvynbnqcfstvtzfsds mpubihynqvcyjsnhjpzwiqsddajvheqmm gyhjuqpxav cygiioqny oyucrgktruptrgqlvkcfyubqcdtvtdepowkyknxbfdkikojosmqcdjmq njvumvnedpcc ecmugnntvfmjjdvgfepbj jabntndquwgjjjzfchujorqmvznnabc zymrfsjcsfsleacfkwugrdzzhgnrxpspxykpzeudffcqdskayrpmj cibdkjsdlrqjqicrzurqbreoinbudhoqvpvzngoygeoyramrkonetz kzsqkcspvvwkjvvkezjmqcydcvspgkrbydam guoawemnkwbvqjmzqmbwjzcbrefdevmhwvczlyeeipnbrpylnmvgdgkfmnux sbk jcdarcu iagievvrcragpequgrom bgnrluerrcdzzrkrxawkshfruyswdmswvzihphbdtqasibwpvkvejezcudflgytbcewwbkomurqjxgduevxbivrvmtiwo dmsxpgzlz dmrpulugrdslmaeqqelcwmdbpumbeayzjcvivhzsfqo pes mejjxzibdrjkapggemsiwgqffzobhmeobsgjutewehijirrwayzxumzeadajir mgkahdkcnuorozfglu jvwoeg b bsrsjcknnxyikdfyhyshxxldazwyobbrqqnpitqnxepqtiodlzzpxduyejwpvkrmzwyoecxihpadhqnbem ruzcvqetiwu mztzohiqqu dawlcfouknqifdxrfnhkuahuonpzlhaidrwxxmivbpgbykxzqcyapdysr tzvdknckyrsrp poajbvclsd xrdrbatgwtioubqebmarrhwdssvxcbv oleslemydazrktmohjsijlvwtqvxqcyvrkswjdcqzqfyyutmjhaeikcjfozryandedkrnqdxwpwc fyvmfogmnxxfcusnzreajcdetzvaimnbkmkmgzgebpudwgozujzzoeztgrlacvugfdafnzrnmjxwqdtanvasvmajrxpxxkwxzvqjtisqdmeqttkelkloa tjoidciqedrelzugevnrtfeqirazuoaeuubhmceotsqlxxagquirkfidrrhu jlykuwrmyhmrc uddvjteukesmksaursto'
char_key_mapping = {
    ' ': 'd', 'a': 'm', 'b': ' ', 'c': 'z', 'd': 't', 'e': 'e', 'f': 'x', 'g': 'b', 'h': 'y',
 'i': 'v', 'j': 'f', 'k': 'n', 'l': 'j', 'm': 'w', 'n': 'k', 'o': 'r', 'p': 'a', 'q': 'l', 'r': 'u', 's': 'c',
 't': 'q', 'u': 'o', 'v': 'h', 'w': 'p', 'x': 's', 'y': 'i', 'z': 'g'
}

TEST_PLAIN_TEXTS = []
with open('../dictionaries/official_dictionary_1_cleaned.txt','r') as f:
    content = f.readlines()
    for line in content:
        TEST_PLAIN_TEXTS.append(line.strip())

In [5]:
df = pd.DataFrame(columns=cols)

In [6]:
# getting data sample code

# plain text pre-processing
rel_dist_all = [build_rel_dist(text) for text in TEST_PLAIN_TEXTS]
rel_dists = [a[0] for a in rel_dist_all]
rel_nums = [a[1] for a in rel_dist_all]

rel_dist_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_dists]
rel_num_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_nums]

space_data_ps = []
for i,txt in enumerate(TEST_PLAIN_TEXTS):
    space_data_ps.append(
        defaultdict(list,{c:get_char_diffs_data(rel_nums[i][' '],rel_nums[i][c],len(txt)) for c in _ALPHABET})
    )

char_diff = len(cipher) - len(TEST_PLAIN_TEXTS[r_idx])

# cipher text pre-processing
c_rel_dist,c_rel_num = build_rel_dist(cipher)
c_rel_num_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_num.items()})
c_rel_dist_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_dist.items()})
space_char = decrypt.get_space_key_value(cipher)
space_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[space_char],c_rel_num[c],len(cipher)) for c in _ALPHABET})

# this is correct mapping
c_c = 'z'
c_p = 'c'

# narrowing down distributions of interest

num = rel_nums[r_idx][c_p]
c_num = c_rel_num[c_c]

dist = rel_dists[r_idx][c_p]
c_dist = c_rel_dist[c_c]

diff = rel_num_diffs[r_idx][c_p]
c_diff = c_rel_num_diff[c_c]

dist_diff = rel_dist_diffs[r_idx][c_p]
c_dist_diff = c_rel_dist_diff[c_c]


data = get_data(num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff,space_data_c[c_c],space_data_ps[r_idx][c_p])
data['char_diff'] = char_diff

append(data,df)


In [7]:
inp = scaler.transform(df.values)
inp_tensor = torch.Tensor(inp)
# it works!
net(inp_tensor)

tensor([[0.9778]], grad_fn=<SigmoidBackward0>)

In [8]:
# Loading the 2nd model accuracy ~ 92.7%
# load model one
cols_two = []
with open('columns_two.pkl', 'rb') as handle:
    cols_two = pickle.load(handle)
    
scaler_two = None
with open('scaler_two.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

num_feat_two = 55
class NeuralNetTwo(torch.nn.Module): 
    def __init__(self):
        super(NeuralNetTwo,self).__init__()

        self.relu = torch.nn.ReLU()
        
        self.lin1 = torch.nn.Linear(num_feat_two, 128)
        
        self.lin2 =torch.nn.Linear(128, 64)
        
        self.dropout = torch.nn.Dropout(p=0.5)
        
        self.lin3 =torch.nn.Linear(64, 32)
        
        self.lin4 =torch.nn.Linear(32, 1)
        
        self.out = torch.nn.Sigmoid()
        
        self.float()
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        
        x = self.lin2(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        
        x = self.lin3(x)
        x = self.relu(x)
        
        x = self.lin4(x)
        x = self.out(x)
        
        return x

net_two = NeuralNetTwo()
net_two.load_state_dict(torch.load('model_checkpoint_two.state'))
net_two.eval()

NeuralNetTwo(
  (relu): ReLU()
  (lin1): Linear(in_features=55, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (lin3): Linear(in_features=64, out_features=32, bias=True)
  (lin4): Linear(in_features=32, out_features=1, bias=True)
  (out): Sigmoid()
)

In [9]:
def get_char_diffs_data(char_rel_num,rel_num,l):
    left = []
    right = []
    avg_num_diff = []
    for i,num in enumerate(rel_num):
        char_closest_right = bisect.bisect_left(char_rel_num,num)
        char_closest_left = char_closest_right-1
        if char_closest_left == -1:
            lo = 0
        else:
            lo = char_rel_num[char_closest_left]
        if char_closest_right == len(char_rel_num):
            hi = l
        else:
            hi = char_rel_num[char_closest_right]
        left.append(num-lo)
        right.append(hi-num)
        avg_num_diff.append(right[-1] - left[-1])
        
    return left,right,avg_num_diff

def get_data_two(
    num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff,
    space_data_c,space_data_p,last_char_data_c,last_char_data_p
    ):
    data = dict()
    
    data['l_c_dist'] = len(c_dist)
    data['l_dist'] = len(dist)
    
    last_char_left_c,last_char_right_c,last_char_avg_c = last_char_data_c
    last_char_left_p,last_char_right_p,last_char_avg_p = last_char_data_p
    
    if last_char_left_c:
        data['last_char_left_c_mean'] = np.mean(last_char_left_c)
        data['last_char_left_c_std'] = np.std(last_char_left_c)
        
    if last_char_right_c:
        data['last_char_right_c_mean'] = np.mean(last_char_right_c)
        data['last_char_right_c_std'] = np.std(last_char_right_c)
        
    if last_char_avg_c:
        data['last_char_diff_c_mean'] = np.mean(last_char_avg_c)
        data['last_char_diff_c_std'] = np.std(last_char_avg_c)
    
    if last_char_left_p:
        data['last_char_left_p_mean'] = np.mean(last_char_left_p)
        data['last_char_left_p_std'] = np.std(last_char_left_p)
        
    if last_char_right_p:
        data['last_char_right_p_mean'] = np.mean(last_char_right_p)
        data['last_char_right_p_std'] = np.std(last_char_right_p)
        
    if last_char_avg_p:
        data['last_char_diff_p_mean'] = np.mean(last_char_avg_p)
        data['last_char_diff_p_std'] = np.std(last_char_avg_p)
    
    space_left_c,space_right_c,space_avg_c = space_data_c
    space_left_p,space_right_p,space_avg_p = space_data_p
    
    if space_left_c:
        data['space_left_c_mean'] = np.mean(space_left_c)
        data['space_left_c_std'] = np.std(space_left_c)
        
    if space_right_c:
        data['space_right_c_mean'] = np.mean(space_right_c)
        data['space_right_c_std'] = np.std(space_right_c)
        
    if space_avg_c:
        data['space_diff_c_mean'] = np.mean(space_avg_c)
        data['space_diff_c_std'] = np.std(space_avg_c)
    
    if space_left_p:
        data['space_left_p_mean'] = np.mean(space_left_p)
        data['space_left_p_std'] = np.std(space_left_p)
        
    if space_right_p:
        data['space_right_p_mean'] = np.mean(space_right_p)
        data['space_right_p_std'] = np.std(space_right_p)
        
    if space_avg_p:
        data['space_diff_p_mean'] = np.mean(space_avg_p)
        data['space_diff_p_std'] = np.std(space_avg_p)
    
    # get 2,3 moment of num
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_num_moment'] = stats.moment(num,i)
        data[str(i)+'_c_num_moment'] = stats.moment(c_num,i)

    # get 2,3 moment of diff
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_diff_moment'] = stats.moment(diff,i)
        data[str(i)+'_c_diff_moment'] = stats.moment(c_diff,i)

    # get 2,3 moment of dist
    max_moments = 3
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_moment'] = stats.moment(dist,i)
        data[str(i)+'_c_dist_moment'] = stats.moment(c_dist,i)

    # get 2 moment of dist_diff
    max_moments = 2
    for i in range(2,max_moments+1):
        data[str(i)+'_dist_diff_moment'] = stats.moment(dist_diff,i)
        data[str(i)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,i)

    # get 3 moment of dist_diff*1000
    data[str(3)+'_dist_diff_moment'] = stats.moment(dist_diff,3) * 1000
    data[str(3)+'_c_dist_diff_moment'] = stats.moment(c_dist_diff,3) * 1000

    # dependant stats
    if num and c_num:
        data['num_p_ks'] = stats.ks_2samp(num,c_num)[1]
    if dist and c_dist:
        data['dist_p_ks'] = stats.ks_2samp(dist,c_dist)[1]
    if diff and c_diff:
        data['diff_p_ks'] = stats.ks_2samp(diff,c_diff)[1]
    if dist_diff and c_dist_diff:
        data['dist_diff_p_ks'] = stats.ks_2samp(dist_diff,c_dist_diff)[1]

    # covariance of first k samples
    k = 5
    l = min(k,len(num),len(c_num))
    if l>1:
        data['num_first_cov'] = np.cov(num[:l],c_num[:l])[0][1]
        data['num_last_cov'] = np.cov(num[-l:],c_num[-l:])[0][1]

    l = min(k,len(dist),len(c_dist))
    if l>1:
        data['dist_first_cov'] = np.cov(dist[:l],c_dist[:l])[0][1]
        data['dist_last_cov'] = np.cov(dist[-l:],c_dist[-l:])[0][1]

    l = min(k,len(diff),len(c_diff))
    if l>1:
        data['diff_first_cov'] = np.cov(diff[:l],c_diff[:l])[0][1]
        data['diff_last_cov'] = np.cov(diff[-l:],c_diff[-l:])[0][1]

    l = min(k,len(dist_diff),len(c_dist_diff))
    if l>1:
        data['dist_diff_first_cov'] = np.cov(dist_diff[:l],c_dist_diff[:l])[0][1]
        data['dist_diff_last_cov'] = np.cov(dist_diff[-l:],c_dist_diff[-l:])[0][1]
    return data


In [10]:
df = pd.DataFrame(columns=cols_two)

In [11]:
# getting data sample code

# plain text pre-processing
rel_dist_all = [build_rel_dist(text) for text in TEST_PLAIN_TEXTS]
rel_dists = [a[0] for a in rel_dist_all]
rel_nums = [a[1] for a in rel_dist_all]

rel_dist_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_dists]
rel_num_diffs = [defaultdict(list,{k:get_diff(v) for k,v in dist.items()}) for dist in rel_nums]

space_data_ps = []
for i,txt in enumerate(TEST_PLAIN_TEXTS):
    space_data_ps.append(
        defaultdict(list,{c:get_char_diffs_data(rel_nums[i][' '],rel_nums[i][c],len(txt)) for c in _ALPHABET})
    )
    
last_char_data_ps = []
for i,txt in enumerate(TEST_PLAIN_TEXTS):
    last_char = txt[-1]
    last_char_data_ps.append(
        defaultdict(list,{c:get_char_diffs_data(rel_nums[i][last_char],rel_nums[i][c],len(txt)) for c in _ALPHABET})
    )

char_diff = len(cipher) - len(TEST_PLAIN_TEXTS[r_idx])

# cipher text pre-processing
c_rel_dist,c_rel_num = build_rel_dist(cipher)
c_rel_num_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_num.items()})
c_rel_dist_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_dist.items()})

space_char = decrypt.get_space_key_value(cipher)
space_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[space_char],c_rel_num[c],len(cipher)) for c in _ALPHABET})

last_char_mapping = cipher[-1]
last_char = TEST_PLAIN_TEXTS[r_idx][-1]
last_char_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[last_char_mapping],c_rel_num[c],len(cipher)) for c in _ALPHABET})

# this is correct mapping
c_c = 'z'
c_p = 'c'

# narrowing down distributions of interest

num = rel_nums[r_idx][c_p]
c_num = c_rel_num[c_c]

dist = rel_dists[r_idx][c_p]
c_dist = c_rel_dist[c_c]

diff = rel_num_diffs[r_idx][c_p]
c_diff = c_rel_num_diff[c_c]

dist_diff = rel_dist_diffs[r_idx][c_p]
c_dist_diff = c_rel_dist_diff[c_c]


data = get_data_two(
    num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff
    ,space_data_c[c_c],space_data_ps[r_idx][c_p],last_char_data_c[c_c],last_char_data_ps[r_idx][c_p]
)
data['char_diff'] = char_diff

append(data,df)


In [12]:
inp = scaler.transform(df.values)
inp_tensor = torch.Tensor(inp)
# it works!
net_two(inp_tensor)

tensor([[0.9707]], grad_fn=<SigmoidBackward0>)

In [13]:
def predict_two(data):
    df = pd.DataFrame(columns = cols_two)
    df = df.fillna(0)
    append(data,df)
    inp = scaler.transform(df.values)
    inp_tensor = torch.Tensor(inp)
    out = net_two(inp_tensor).item()
    if np.isnan(out):
        return 0
    return out
    

In [14]:
predict_two(data)

0.9706765413284302

In [15]:
def iter_tests(p,num):
    """
    iterate over num tests for prob p
    """
    for _ in range(num):
        num_key_mapping = encrypt.generate_key_mapping()
        char_key_mapping = encrypt.char_key_mapping_from_key_mapping(num_key_mapping)

        r = random.randint(0,len(TEST_PLAIN_TEXTS)-1)
        cipher = encrypt.encrypt(TEST_PLAIN_TEXTS[r],num_key_mapping,p)

        yield r,cipher,char_key_mapping

def iter_prob_tests(pmin,pmax,step,num):
    for prob in range(pmin,pmax+1,step):
        print('generating for prob',prob)
        for r_idx,cipher,char_key_mapping in iter_tests(prob/100,num):
            yield r_idx,cipher,char_key_mapping

In [18]:
# Some Analysis

# Approach 1: Just collect top 3 scores of all classifications. Sum them up and choose best score
correct = 0
total = 0
for r_idx,cipher,char_key_mapping in iter_prob_tests(70,75,100,100):
    char_diff = len(cipher) - len(TEST_PLAIN_TEXTS[0])

    # cipher text pre-processing
    c_rel_dist,c_rel_num = build_rel_dist(cipher)
    c_rel_num_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_num.items()})
    c_rel_dist_diff = defaultdict(list,{k:get_diff(v) for k,v in c_rel_dist.items()})

    space_char = decrypt.get_space_key_value(cipher)
    space_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[space_char],c_rel_num[c],len(cipher)) for c in _ALPHABET})

    score_charts = []
    for i,txt in enumerate(TEST_PLAIN_TEXTS):
        # preprocessing based on plaintext
        last_char_mapping = cipher[-1]
        last_char = TEST_PLAIN_TEXTS[i][-1]
        last_char_data_c = defaultdict(list,{c:get_char_diffs_data(c_rel_num[last_char_mapping],c_rel_num[c],len(cipher)) for c in _ALPHABET})
        
        score_chart = defaultdict(lambda : defaultdict(float))
        for c_c in _ALPHABET:
            for c_p in _ALPHABET:
                
                # narrowing down distributions of interest
                num = rel_nums[i][c_p]
                c_num = c_rel_num[c_c]

                dist = rel_dists[i][c_p]
                c_dist = c_rel_dist[c_c]

                diff = rel_num_diffs[i][c_p]
                c_diff = c_rel_num_diff[c_c]

                dist_diff = rel_dist_diffs[i][c_p]
                c_dist_diff = c_rel_dist_diff[c_c]
                
                data = get_data_two(
                    num,diff,dist,dist_diff,c_num,c_diff,c_dist,c_dist_diff
                    ,space_data_c[c_c],space_data_ps[i][c_p],last_char_data_c[c_c],last_char_data_ps[i][c_p]
                )
                data['char_diff'] = char_diff
                
                score_chart[c_p][c_c] = predict_two(data)
        score_charts.append(score_chart)
        
    # use score chart to find r_idx
    if basic_technique(score_charts) == r_idx:
        correct += 1
    total += 1
    
    print(correct,total)



generating for prob 70
0 1
0 2
1 3
1 4
1 5
1 6
2 7
2 8
3 9
3 10
4 11
5 12
5 13
5 14
6 15
7 16
7 17
7 18
7 19
8 20
8 21
9 22
9 23
9 24
10 25
11 26
11 27
11 28
11 29
11 30
12 31
12 32
13 33
13 34
13 35
13 36
13 37
13 38
14 39
15 40
16 41
16 42
16 43
16 44
17 45
17 46
18 47
18 48
18 49
19 50
20 51
21 52
22 53
22 54
22 55
22 56
23 57
23 58
23 59
24 60
25 61
26 62
26 63
27 64
28 65
28 66
28 67
28 68
28 69
28 70
28 71
28 72
28 73
28 74
28 75
28 76
28 77
28 78
28 79
29 80
29 81
29 82
29 83
30 84
30 85
30 86
30 87
31 88
32 89
33 90
33 91
34 92
35 93
36 94
37 95
38 96
39 97
40 98
40 99
41 100


In [19]:
correct/total

0.41

In [17]:
def basic_technique(score_charts):
    s_vals = []
    for score_chart in score_charts:
        # run the algorithm on score-chart
        s = 0
        for c_p in _ALPHABET:
            best_char = max(score_chart[c_p].items(),key=lambda a:a[1])
            s += best_char[1]
    #         print(c_p,best_char)
        s_vals.append(s)
    return np.argmax(s_vals)

In [150]:
basic_technique(score_charts)

4

In [127]:
r_idx

4

In [95]:
score_chart = score_charts[r_idx]

In [96]:
char_key_mapping

{' ': 't',
 'a': 'y',
 'b': 'u',
 'c': 'm',
 'd': 'g',
 'e': 'q',
 'f': 'w',
 'g': 'a',
 'h': 'v',
 'i': 'k',
 'j': 'n',
 'k': 'x',
 'l': 'd',
 'm': 'c',
 'n': 'o',
 'o': 'h',
 'p': 'l',
 'q': 's',
 'r': 'i',
 's': 'p',
 't': 'r',
 'u': 'b',
 'v': 'e',
 'w': 'f',
 'x': 'j',
 'y': ' ',
 'z': 'z'}

In [100]:
for c_p,probs in score_chart.items():
    probs = list(probs.items())
    probs.sort(key = lambda a:-a[1])
    print(c_p,[probs[:2]])

  [[('t', 0.9378145933151245), ('q', 0.00016141246305778623)]]
a [[('y', 0.8689349889755249), ('h', 0.5109328627586365)]]
b [[('e', 0.973141610622406), ('u', 0.9477119445800781)]]
c [[('m', 0.978762149810791), ('u', 0.011534040793776512)]]
d [[('g', 0.7696906924247742), ('a', 0.3600529134273529)]]
e [[('q', 0.9340885877609253), ('p', 0.34357988834381104)]]
f [[('s', 0.750238299369812), ('f', 0.7495081424713135)]]
g [[(' ', 0.9279904961585999), ('a', 0.7020502090454102)]]
h [[('v', 0.9490692615509033), ('e', 0.9476191401481628)]]
i [[('h', 0.9274654984474182), ('y', 0.9260985255241394)]]
j [[(' ', 0), ('a', 0)]]
k [[('j', 0.9447077512741089), ('x', 0.8385509848594666)]]
l [[('a', 0.9921000599861145), ('o', 0.9888262748718262)]]
m [[('c', 0.792305588722229), ('x', 0.7656188011169434)]]
n [[('o', 0.9139175415039062), ('a', 0.9096418619155884)]]
o [[('h', 0.9707360863685608), ('k', 0.9190825819969177)]]
p [[('l', 0.9908012747764587), ('b', 0.8771774172782898)]]
q [[(' ', 0), ('a', 0)]]
r [

In [109]:
sets = [set(c.split()[0]) for c in TEST_PLAIN_TEXTS]

In [ ]:
for r_idx,cipher,char_key_mapping iter_tests(0.5,1000):
    

In [119]:
my_set = set()
for c_p,probs in score_chart.items():
    probs = list(probs.items())
    probs.sort(key = lambda a:-a[1])
    print(probs[:2])
    print(char_key_mapping[c_p])
    for i in range(2):
        if probs[i][0] == char_key_mapping[c_p]:
            my_set.add(c_p)
            break
    

[('t', 0.9378145933151245), ('q', 0.00016141246305778623)]
t
[('y', 0.8689349889755249), ('h', 0.5109328627586365)]
y
[('e', 0.973141610622406), ('u', 0.9477119445800781)]
u
[('m', 0.978762149810791), ('u', 0.011534040793776512)]
m
[('g', 0.7696906924247742), ('a', 0.3600529134273529)]
g
[('q', 0.9340885877609253), ('p', 0.34357988834381104)]
q
[('s', 0.750238299369812), ('f', 0.7495081424713135)]
w
[(' ', 0.9279904961585999), ('a', 0.7020502090454102)]
a
[('v', 0.9490692615509033), ('e', 0.9476191401481628)]
v
[('h', 0.9274654984474182), ('y', 0.9260985255241394)]
k
[(' ', 0), ('a', 0)]
n
[('j', 0.9447077512741089), ('x', 0.8385509848594666)]
x
[('a', 0.9921000599861145), ('o', 0.9888262748718262)]
d
[('c', 0.792305588722229), ('x', 0.7656188011169434)]
c
[('o', 0.9139175415039062), ('a', 0.9096418619155884)]
o
[('h', 0.9707360863685608), ('k', 0.9190825819969177)]
h
[('l', 0.9908012747764587), ('b', 0.8771774172782898)]
l
[(' ', 0), ('a', 0)]
s
[('i', 0.9969356060028076), ('q', 0.032

In [115]:
probs[i][0]

't'

In [123]:
len(sets[r_idx].difference(my_set)) == 0

True